In [1]:
from yolo import yolo
from save import SaveNumpy

In [2]:
image = r"C:\Users\JiraponSasomsap\ArticulusProjects\projects\PTG\old\punthai\panthai_videos\cam2.jpg"

In [3]:
model_path = r"C:\Users\JiraponSasomsap\ArticulusProjects\public_models\yolov8x-pose-p6.pt"
model = yolo(model_path).set_config(image)

In [4]:
rsave = SaveNumpy(save_as='.')
results = model.predict_and_save(image, verbose = False)

save_parent = rsave.npy(results[0].boxes.xyxy.cpu().numpy(), model._config)

In [5]:
rsave.save_config(model._config, save_parent.parent.parent)

In [6]:
# 